In [68]:
#- 패키지 임포트
import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Okt
okt = Okt()
from collections import Counter # 형태소별 빈도 구할 때 사용

In [69]:
''' 파일 불러오기 '''
df = pd.read_csv("../../Sentiment_analysis/Primary_classification/Ternary_classification_2018.csv", encoding='UTF-8')
print(df.shape, df.columns)
df

(135824, 3) Index(['sentence', 'predicted_sentiment', 'org_idx'], dtype='object')


,sentence,predicted_sentiment,org_idx
0,블로그 mom 5개의 글 mom 목록 열기 영어 명언 모음입니다.,긍정,0
1,"mom 2018, 12, 31, 19, 52 https blog NAVER com ...",긍정,0
2,An enemy generally says and believes what he w...,긍정,0
3,True love is the joy of life 진실한 사랑은 인생의 환희다,긍정,0
4,Crape idem 현재를 즐겨라.,긍정,0
...,...,...,...
135819,한편 경계의 의미로 이 사자성어를 추천한 이들도 눈에 띈다.,긍정,5170
135820,조은영 원광대 교수 미술과는 2017년을 종합하기에는 수락 석출 외의 단어들이 지나...,긍정,5170
135821,올해의 사자성어는 3위부터 5위까지는 약 16대의 고른 분포를 보인 것이 특징이다.,긍정,5170
135822,"4위는 16, 5위는 15, 1 쳤다",부정,5170


In [70]:
for emo in ['긍정','부정','중립']:

    ''' 형태소 분석 '''
    nouns = []
    verbs = []
    adjs = []

    df_class = df[df['predicted_sentiment'] == "%s"%emo]
    corpus = df_class['sentence'].to_list()

    for sent in tqdm(range(len(corpus))):
        # 문장에서 형태소/품사 추출
        try:
            a = okt.pos(corpus[sent], norm=True, stem=True) # 단어의 정규화와 어간 추출을 실행(True).
            for x, y in a:
                # 품사가 명사면 명사 리스트에 단어 추가
                if y == 'Noun':
                    nouns.append(x)
                # 품사가 동사면 동사 리스트에 단어 추가
                elif y == 'Verb':
                    verbs.append(x)
                # 품사가 형용사면 형용사 리스트에 단어 추가
                elif y == 'Adjective':
                    adjs.append(x)
        except:
            pass

    morph_dic = {'명사':nouns, '동사':verbs, '형용사':adjs}
    for mor in ['명사','동사','형용사']:
        
        # 명사 리스트에서 명사 빈도 리스트 생성
        morph_cnt = Counter(morph_dic[mor]).most_common()

        morph = []
        morph_freq = []
        for key, val in morph_cnt:
            morph.append(key)
            morph_freq.append(val)

        # 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
        morph_df = pd.DataFrame({'morph':morph, 'morph_freq':morph_freq})
        morph_df.to_csv("2018_감정예측_%s_%s.csv"%(emo, mor), index=False, encoding='UTF-8')

100%|██████████| 148/148 [00:00<00:00, 329.76it/s]


In [142]:
for year in range(2018,2023):
    for emo in ['긍정','부정','중립']:
        df1 = pd.read_csv("%s_감정예측_%s_형용사.csv"%(str(year), emo), encoding='UTF-8')
        df2 = pd.read_csv("%s_감정예측_%s_동사.csv"%(str(year), emo), encoding='UTF-8')
        df3 = pd.read_csv("%s_감정예측_%s_명사.csv"%(str(year), emo), encoding='UTF-8')

        #  데이터프레임 concat
        df_concat = pd.concat([df1,df2,df3],axis=1)
        # column 매핑 생성
        column_mapping = {
            'morph': '형용사',
            'morph.1': '동사',
            'morph.2': '명사',
            'morph_freq': '빈도',
            'morph_freq.1': '빈도',
            'morph_freq.2': '빈도',
        }

        # 데이터프레임의 column명을 매핑으로 바꾸기 (원본 데이터프레임 변경)
        df_concat.rename(columns=column_mapping, inplace=True)
        df_concat.to_excel("%s_감정예측_%s_단어종합.xlsx"%(year, emo), index=False, encoding='UTF-8')

In [71]:
import plotly.express as px

# unique 값들의 개수를 계산
sentiment_counts = df['predicted_sentiment'].value_counts()


# 막대그래프로 시각화
fig = px.bar(sentiment_counts, x=sentiment_counts.index, y=sentiment_counts.values,
             labels={'x': 'Sentiment', 'y': 'Count'},
             title='Sentiment Distribution 2018',
             color=sentiment_counts.index,
             text=sentiment_counts.values)  # 개수를 막대에 표시하기 위해 text 속성에 sentiment_counts.values 할당

# 개수를 막대 위에 표시하는 옵션 추가
fig.update_traces(texttemplate='%{text}', textposition='outside')

# y축 범위 설정 (기존 최대값의 10% 더 큰 값으로 설정)
y_axis_range = sentiment_counts.max() * 1.1
fig.update_layout(yaxis_range=[0, y_axis_range])

# y축 값의 형식 설정 (정수 형식으로 표시)
fig.update_layout(yaxis_tickformat=',d')

# 글꼴 크기 설정
fig.update_layout(
    font=dict(
        family="Arial, bold",
        size=13
    )
)

# x축 레이블 설정
fig.update_xaxes(title_text='Label')

# 제목을 가운데에 위치시키기
fig.update_layout(title_x=0.5)

# 그래프 출력
fig.show()

In [72]:
df_1 = pd.read_csv("../../Sentiment_analysis/Primary_classification/Ternary_classification_2018.csv", encoding='UTF-8')
df_2 = pd.read_csv("../../Sentiment_analysis/Primary_classification/Ternary_classification_2019.csv", encoding='UTF-8')
df_3 = pd.read_csv("../../Sentiment_analysis/Primary_classification/Ternary_classification_2020.csv", encoding='UTF-8')
df_4 = pd.read_csv("../../Sentiment_analysis/Primary_classification/Ternary_classification_2021.csv", encoding='UTF-8')
df_5 = pd.read_csv("../../Sentiment_analysis/Primary_classification/Ternary_classification_2022.csv", encoding='UTF-8')

In [73]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 각 데이터프레임의 연도 정보 추가
df_1['year'] = 2018
df_2['year'] = 2019
df_3['year'] = 2020
df_4['year'] = 2021
df_5['year'] = 2022

# 데이터프레임들을 합치기
merged_df = pd.concat([df_1, df_2, df_3, df_4, df_5])

# 연도별로 그룹화하여 unique 값 개수 계산
grouped_df = merged_df.groupby(['year', 'predicted_sentiment']).size().unstack(fill_value=0)

# Subplots 생성
fig = make_subplots(rows=1, cols=3, subplot_titles=('긍정', '부정', '중립'))

# 각 subplot에 데이터 추가 (선 그래프 + 툴팁)
fig.add_trace(go.Scatter(x=grouped_df.index, y=grouped_df['긍정'], mode='markers+lines', name='긍정',
                         text=grouped_df['긍정'], textposition='top center', hoverinfo='x+y+text'), row=1, col=1)
fig.add_trace(go.Scatter(x=grouped_df.index, y=grouped_df['부정'], mode='markers+lines', name='부정',
                         text=grouped_df['부정'], textposition='top center', hoverinfo='x+y+text'), row=1, col=2)
fig.add_trace(go.Scatter(x=grouped_df.index, y=grouped_df['중립'], mode='markers+lines', name='중립',
                         text=grouped_df['중립'], textposition='top center', hoverinfo='x+y+text'), row=1, col=3)

# 레이아웃 설정
fig.update_layout(
    title='Sentiment transition curve by Year',
    title_x=0.5,
    font=dict(family='Arial, bold', size=14)
)
# Y축의 형식을 정수로 설정
fig.update_yaxes(tickformat='d')

fig.update_xaxes(title_text="Year", row=1, col=1)
fig.update_xaxes(title_text="Year", row=1, col=2)
fig.update_xaxes(title_text="Year", row=1, col=3)

fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=2)
fig.update_yaxes(title_text="Count", row=1, col=3)

# X축의 눈금 값을 2018, 2019, 2020으로 설정
fig.update_xaxes(tickvals=[2018, 2019, 2020, 2021, 2022])

# 그래프 출력
fig.show()

In [83]:
df_5['predicted_sentiment'].value_counts()

긍정    211903
부정     50238
중립       167
Name: predicted_sentiment, dtype: int64

In [84]:
len(df_5)

262308

In [85]:
[round((i/len(df_5))*100) for i in df_5['predicted_sentiment'].value_counts()]

[81, 19, 0]

In [78]:
data = [
     [round((i/len(df_5))*100) for i in df_5['predicted_sentiment'].value_counts()],
     [round((i/len(df_4))*100) for i in df_4['predicted_sentiment'].value_counts()],
     [round((i/len(df_3))*100) for i in df_3['predicted_sentiment'].value_counts()],
     [round((i/len(df_2))*100) for i in df_2['predicted_sentiment'].value_counts()],
     [round((i/len(df_1))*100) for i in df_1['predicted_sentiment'].value_counts()],
          ]
data

[[81, 19, 0], [82, 18, 0], [83, 17, 0], [83, 17, 0], [81, 19, 0]]

In [80]:
for i in range(len(data[0])):
    lst = []
    for a in range(len(data)):
        lst.append(data[a][i])
    print(lst)

[81, 82, 83, 83, 81]
[19, 18, 17, 17, 19]
[0, 0, 0, 0, 0]


In [81]:
import plotly.graph_objects as go

colors = ['rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)',
          'rgba(122, 120, 168, 0.8)']
          
# 데이터
years = ['2022\t', '2021\t', '2020\t', '2019\t', '2018\t']
positives = [81, 82, 83, 83, 81]
negatives = [19, 18, 17, 17, 19]
neutrals = [0, 0, 0, 0, 0]

# 수평 누적 막대 그래프 생성
fig = go.Figure()

# 긍정, 부정, 중립 막대 그래프 추가 (서로 다른 색상 적용)
fig.add_trace(go.Bar(y=years, x=positives, text=[str(pos) + "%" + "\t"*60 for pos in positives], textposition='inside', name='긍정', orientation='h', marker=dict(color='rgba(38, 24, 74, 0.8)')))
fig.add_trace(go.Bar(y=years, x=negatives, text=[str(neg) + "%" + "\t"*16 for neg in negatives], textposition='inside', name='부정', orientation='h', marker=dict(color='rgba(71, 58, 131, 0.8)')))
fig.add_trace(go.Bar(y=years, x=neutrals, text=[str(neu) + "%" + "\t"*3 for neu in neutrals], textposition='inside', name='중립', orientation='h', marker=dict(color='rgba(122, 120, 168, 0.8)')))

# 그래프 레이아웃 설정
fig.update_layout(
    barmode='stack',
    title='연도별 긍정/부정/중립 비율',
    title_x=0.47,  # 제목 가운데 정렬
    xaxis_title='비율 (%)',
    yaxis_title='',
    legend=dict(orientation='v', xanchor='right', yanchor='bottom', x=1.05, y=0),  # 오른쪽 아래로 위치 변경
    margin=dict(l=200, r=400, t=140, b=80),  # 좌우너비 조정
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
    font=dict(family='Arial', size=14, color='rgb(67, 67, 67)'),
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.15, 1]
    ),
)

fig.show()